## Ampliando o conhecimento de LLMs com dados externos

Este notebook é um material de apoio para a oficina, apresentando um exemplo de implementação de uma arquitetura usando a abordagem Retrieval-Augmented Generation (RAG). O objetivo é demonstrar como é possível ampliar o conhecimento de Large Language Models (LLMs) a partir de documentos proprietários, melhorando a precisão nas respostas e reduzindo possíveis alucinações sobre temas específicos. 

O notebook possui as seguintes seções: </br>
1 - Importação de bibliotecas e setup do ambiente </br>
2 - Exemplo de interação com um LLM pré-treinado </br>
3 - Exemplo de criação de uma vector store, possibilitando buscas semânticas em documentos </br>
4 - Exemplo de interação com um LLM pré-treinado através de uma arquitetura RAG </br>

### 1 - Importação de bibliotecas e setup do ambiente

Importação das bibliotecas necessárias para executar este notebook.

In [1]:
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from openai import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.vectorstores import FAISS

load_dotenv()

True

### 2 - Exemplo de interação com um LLM pré-treinado

Neste exemplo, primeiramente avaliamos o conhecimento do modelo pré-treinado "gpt-4o-mini".

In [2]:
client = OpenAI()

Primeiramente, avaliamos o conhecimento do modelo sobre Alan Turing. Podemos observar que o modelo responde sobre quem é Alan Turing sem maiores dificuldades.

In [3]:

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "Você é um assistente para responder perguntas"},
    {"role": "user", "content": "Quem é Alan Turing?"}
  ]
)

response.choices[0].message.content

'Alan Turing foi um matemático, lógico, filósofo e criptógrafo britânico, amplamente reconhecido como um dos pais da ciência da computação moderna. Nascido em 23 de junho de 1912, em Londres, Turing fez contribuições significativas para a teoria da computação, incluindo a definição da "máquina de Turing", um conceito fundamental que ajuda a entender o que significa calcular.\n\nDurante a Segunda Guerra Mundial, Turing trabalhou para os serviços de inteligência britânicos no Bletchley Park, onde desempenhou um papel crucial na decodificação de mensagens criptografadas dos nazistas, particularmente aquelas codificadas pela máquina Enigma. Seu trabalho foi vital para a vitória dos Aliados e ajudou a encurtar a guerra.\n\nAlém de suas contribuições à criptografia e à matemática, Turing também é lembrado por seu trabalho em inteligência artificial e pela proposta do "Teste de Turing", um critério para avaliar a inteligência de máquinas.\n\nTuring enfrentou discriminação devido à sua homosse

Porém, ao ser questionado sobre uma pessoa menos conhecida, o modelo afirma que não possui conhecimentos suficientes para responder a pergunta. Isso ocorre porque o modelo não adquiriu esse conhecimento durante o seu treinamento.

In [4]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": "Você é um assistente para responder perguntas"},
    {"role": "user", "content": "Quem é Wesllei Heckler?"}
  ]
)

response.choices[0].message.content

'Wesllei Heckler é uma figura pública conhecida no Brasil, principalmente por seu trabalho no mundo do marketing digital e como influenciador nas redes sociais. Ele se destaca por compartilhar dicas sobre empreendedorismo, estratégias de venda e como alavancar negócios online. No entanto, como as informações sobre figuras públicas podem mudar rapidamente, recomendo verificar fontes atualizadas para obter os detalhes mais recentes sobre ele. Se você tiver uma pergunta mais específica, sinta-se à vontade para perguntar!'

Uma possível estratégia para contornar esse problema é fornecer dados sobre a pessoa desconhecida. Desta forma, o modelo consegue responder a pergunta normalmente, pois possui uma excelente habilidade de localizar informações em textos. 

In [5]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "system", "content": """
        Você é um assistente para responder perguntas. 
        A cada interação, você receberá um contexto e uma pergunta.
        Utilize o contexto para responder a pergunta.
     """},
    {"role": "user", "content": """
        Contexto:
        Wesllei é doutorando no Programa de Pós-Graduação em Computação Aplicada (PPGCA) da Universidade do Vale do Rio dos Sinos
        (UNISINOS). Possui Mestrado em Computação Aplicada na mesma universidade (2022) e graduação em Ciência da Computação pela
        Universidade Feevale (2019). Atua como Cientista de Dados, desenvolvendo soluções de dados através do uso de Ciência de Dados e
        Inteligência Artificial, com experiência no desenvolvimento, validação e monitoramento de modelos de Machine Learning. Wesllei
        também é pesquisador voluntário no Laboratório de Computação Móvel (Mobilab) da Universidade do Vale do Rio dos Sinos
        (UNISINOS), onde colabora em pesquisas que envolvem a aplicação de tecnologias na área da saúde, principalmente envolvendo
        Machine Learning. Atualmente pesquisa modelos de Machine Learning e Inteligência Artificial Generativa para a área de Saúde
        Mental. Seus principais interesses de pesquisa são Data Science, Machine Learning e tecnologia aplicados à Saúde Mental. 
     
        Pergunta:
        Quem é Wesllei Heckler?
     """},
  ]
)

print(response.choices[0].message.content)

Wesllei Heckler é doutorando no Programa de Pós-Graduação em Computação Aplicada (PPGCA) da Universidade do Vale do Rio dos Sinos (UNISINOS). Ele possui um Mestrado em Computação Aplicada na mesma universidade, obtido em 2022, e uma graduação em Ciência da Computação pela Universidade Feevale, concluída em 2019. Atua como Cientista de Dados, desenvolvendo soluções por meio da Ciência de Dados e Inteligência Artificial, com experiência no desenvolvimento, validação e monitoramento de modelos de Machine Learning. Além disso, Wesllei é pesquisador voluntário no Laboratório de Computação Móvel (Mobilab) da UNISINOS, onde participa de pesquisas que aplicam tecnologias na área da saúde, com um foco especial em Machine Learning. Atualmente, ele investiga modelos de Machine Learning e Inteligência Artificial Generativa voltados para a Saúde Mental, sendo seus principais interesses de pesquisa em Data Science, Machine Learning e tecnologia aplicada à Saúde Mental.


### 3 - Exemplo de criação de uma vector store, possibilitando buscas semânticas em documentos

Uma vector store é uma base de dados semântica (ou banco de dados vetorial), que armazena documentos segmentados em forma de vetores. Esses vetores também são conhecidos como embeddings e são gerados por um modelo de Inteligência Artificial (IA) adicional. 

Existem diversas ferramentas para gerenciar vector stores. Neste exemplo, vamos utilizar a FAISS, uma biblioteca da Meta que possibilita a criação de vector stores localmente, fornecendo funções de busca semântica. Nas células abaixo, vamos criar uma vector store para armazenar um documento pdf, segmentando o texto em chunks de 1000 caracteres.

Links úteis:
- [FAISS](https://ai.meta.com/tools/faiss/)
- [Integração da FAISS com LangChain](https://python.langchain.com/v0.2/docs/integrations/vectorstores/faiss/)

In [6]:
def split_paragraphs(rawText):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len,
        is_separator_regex=False,
    )

    return  text_splitter.split_text(rawText)

def load_pdfs(documents_path, pdfs):
    text_chunks = []

    for pdf in pdfs:
        reader = PdfReader(f"{documents_path}/{pdf}")
        for page in reader.pages:
            raw = page.extract_text()
            chunks = split_paragraphs(raw)
            text_chunks += chunks
    return text_chunks

In [7]:
list_of_pdfs = ["documento.pdf"]
text_chunks = load_pdfs('documentos', list_of_pdfs)
text_chunks[0]

'Nome\nWesllei Felipe Heckler\nNome em citações\nbibliográficas\nHECKLER, W. F.;FELIPE HECKLER, WESLLEI;HECKLER,\nWESLLEI;HECKLER, WESLLEI F .;HECKLER, WESLLEI FELIPE\nLattes iD\nhttp://lat tes.cnpq.br/5020012898313017Wesllei Felipe Heckler\nEnder eço par a acessar este CV : http://lat tes.cnpq.br/5020012898313017\nID Lat tes: 5020012898313017\nÚltima atual ização do currículo em 09/09/2024\nWesllei é doutor ando no Programa de Pós-Gr aduação em Computação Aplicada (PPGCA) da Universidade do Vale do Rio dos Sinos\n(UNISINOS). Possui Mestr ado em Computação Aplicada na mesma universidade (2022) e graduação em Ciência da Computação pela\nUniversidade Feevale (2019). Atua como Cientista de Dados, desen volvendo soluções de dados através do uso de Ciência de Dados e\nInteligência Artificial, com experiência no desenvolvimento , validação e monitoramento  de modelos de Machine Learning. Wesllei'

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
faiss_vector_store = FAISS.from_texts(text_chunks, embeddings)

Com a vector store criada, podemos realizar um processo de busca semântica, a fim de encontrar segmentos semanticamente similares a um termo. Podemos configurar o número de segmentos retornados para cada busca.

In [12]:
results = faiss_vector_store.similarity_search(
    "Wesllei Heckler",
    k=1
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
    print()

* Nome
Wesllei Felipe Heckler
Nome em citações
bibliográficas
HECKLER, W. F.;FELIPE HECKLER, WESLLEI;HECKLER,
WESLLEI;HECKLER, WESLLEI F .;HECKLER, WESLLEI FELIPE
Lattes iD
http://lat tes.cnpq.br/5020012898313017Wesllei Felipe Heckler
Ender eço par a acessar este CV : http://lat tes.cnpq.br/5020012898313017
ID Lat tes: 5020012898313017
Última atual ização do currículo em 09/09/2024
Wesllei é doutor ando no Programa de Pós-Gr aduação em Computação Aplicada (PPGCA) da Universidade do Vale do Rio dos Sinos
(UNISINOS). Possui Mestr ado em Computação Aplicada na mesma universidade (2022) e graduação em Ciência da Computação pela
Universidade Feevale (2019). Atua como Cientista de Dados, desen volvendo soluções de dados através do uso de Ciência de Dados e
Inteligência Artificial, com experiência no desenvolvimento , validação e monitoramento  de modelos de Machine Learning. Wesllei [{}]



In [13]:
results = faiss_vector_store.similarity_search(
    "Feevale",
    k=3
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")
    print()

* Vínculo: Celetista, Enquadr amento Funcional: Coordenador
de Suporte, Car ga horária: 40
Outras informações
Coordenação da equipe de suporte técnico , suporte técnico
ao cliente e r ealização de visi tas a cl ientes.
Vínculo institucional
2012 - 2014
Vínculo: Celetista, Enquadr amento Funcional: Suporte
Técnico , Carga horária: 40
Outras informações
Suporte técnico para sistemas de departamento pessoal e
RH, escri ta fiscal, contabi lidade e gestão empr esarial.
Vínculo institucional
2018 - 2020
Vínculo: Colabor ador, Enquadr amento Funcional:
Pesquisador de Aperf eiçoamento Científico
Vínculo institucionalUniversidade Feevale, FEEVALE, Brasil.
CWI Software LTDA, CWI, Brasil. [{}]

* Feevale (INOV AMUNDI), 2018. p . 78-78.
1.
PFEIFFER SALOMÃO DIAS, LUCA S ; DAMA SCENO VIANNA, HENRIQUE ; FELIPE
HECKLER, WESLLEI  ; BARB OSA, JORG E LUIS VICTÓRIA . Identificando Comportamentos
de Risco para Doenças Crônicas: Uma abordagem baseada em ontologia . iSys - Revista
Brasileira de Sistemas de I

### 4 - Exemplo de interação com um LLM pré-treinado através de uma arquitetura RAG

A partir da criação da vector store, podemos automatizar o processo de enriquecimento de contexto ao interagir com LLMs. Na prática, a arquitetura RAG inclui uma camada de busca semântica anterior à interação com o LLM, a fim de buscar documentos possivelmente relevantes para o processamento do prompt. 

In [14]:
retriever = faiss_vector_store.as_retriever(search_type="mmr",
                                            search_kwargs={"k": 1})
llm = ChatOpenAI(model="gpt-4o-mini")

É necessário definir um template de prompt, no qual o system prompt refere-se ao papel que o LLM deve interpretar ao processar o prompt do usuário, seguindo regras específicas. 

In [15]:
system_prompt = (
    "Você é um assistente para responder perguntas. "
    "Use os seguintes segmentos de contextos recuperados para responder"
    "a pergunta. Se você não souber a resposta, diga que não sabe "
    "Use no máximo três sentenças para responder, a fim de manter"
    "a resposta concisa."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

A biblioteca LangChain é a mais popular para o desenvolvimento de sistemas baseados em IA Generativa, disponibilizando componentes pré-configurados que facilitam a implementação desses sistemas.

Link útil:
- [Tutorial de RAG com LangChain](https://python.langchain.com/v0.2/docs/tutorials/rag/)

In [16]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

Com a cadeia criada, agora o sistema busca automaticamente o contexto para responder à nossa pergunta inicial sobre uma pessoa desconhecida para o LLM pré-treinado. 

In [17]:
response = rag_chain.invoke({"input": "Quem é Wesllei Heckler?"})
print('Resposta:')
print(response["answer"])

print()
print('Contexto:')
for document in response["context"]:
    print(document.page_content)
    print()

Resposta:
Wesllei Felipe Heckler é um doutorando no Programa de Pós-Graduação em Computação Aplicada da Universidade do Vale do Rio dos Sinos (UNISINOS). Ele possui mestrado em Computação Aplicada e graduação em Ciência da Computação. Atua como Cientista de Dados, focando em soluções de dados e desenvolvimento de modelos de Machine Learning.

Contexto:
Nome
Wesllei Felipe Heckler
Nome em citações
bibliográficas
HECKLER, W. F.;FELIPE HECKLER, WESLLEI;HECKLER,
WESLLEI;HECKLER, WESLLEI F .;HECKLER, WESLLEI FELIPE
Lattes iD
http://lat tes.cnpq.br/5020012898313017Wesllei Felipe Heckler
Ender eço par a acessar este CV : http://lat tes.cnpq.br/5020012898313017
ID Lat tes: 5020012898313017
Última atual ização do currículo em 09/09/2024
Wesllei é doutor ando no Programa de Pós-Gr aduação em Computação Aplicada (PPGCA) da Universidade do Vale do Rio dos Sinos
(UNISINOS). Possui Mestr ado em Computação Aplicada na mesma universidade (2022) e graduação em Ciência da Computação pela
Universidade Fe

Conforme as instruções definidas no system_prompt, o sistema indica quando não possui informações suficientes para responder uma pergunta.

In [18]:
response = rag_chain.invoke({"input": "Onde Wesllei Heckler mora?"})
print('Resposta:')
print(response["answer"])

print()
print('Contexto:')
for document in response["context"]:
    print(document.page_content)
    print()

Resposta:
Não sei onde Wesllei Heckler mora.

Contexto:
Nome
Wesllei Felipe Heckler
Nome em citações
bibliográficas
HECKLER, W. F.;FELIPE HECKLER, WESLLEI;HECKLER,
WESLLEI;HECKLER, WESLLEI F .;HECKLER, WESLLEI FELIPE
Lattes iD
http://lat tes.cnpq.br/5020012898313017Wesllei Felipe Heckler
Ender eço par a acessar este CV : http://lat tes.cnpq.br/5020012898313017
ID Lat tes: 5020012898313017
Última atual ização do currículo em 09/09/2024
Wesllei é doutor ando no Programa de Pós-Gr aduação em Computação Aplicada (PPGCA) da Universidade do Vale do Rio dos Sinos
(UNISINOS). Possui Mestr ado em Computação Aplicada na mesma universidade (2022) e graduação em Ciência da Computação pela
Universidade Feevale (2019). Atua como Cientista de Dados, desen volvendo soluções de dados através do uso de Ciência de Dados e
Inteligência Artificial, com experiência no desenvolvimento , validação e monitoramento  de modelos de Machine Learning. Wesllei



Podemos observar que o sistema segue aproveitando o conhecimento obtido pelo LLM durante o seu treinamento. Apesar de não termos documentos sobre Alan Turing na vector store criada, o sistema ainda é capaz de responder sobre quem é Alan Turing. 

Também é possível notar que, ainda assim, o sistema considera um segmento de documento para responder a pergunta. Porém, o LLM ignora esse contexto ao entender que o mesmo não é relevante para a resposta final.

In [19]:
response = rag_chain.invoke({"input": "Quem é Alan Turing?"})
print('Resposta:')
print(response["answer"])

print()
print('Contexto:')
for document in response["context"]:
    print(document.page_content)
    print()

Resposta:
Alan Turing foi um matemático, lógico e criptógrafo britânico, considerado um dos pais da ciência da computação e da inteligência artificial. Ele é conhecido por desenvolver a máquina de Turing, um conceito fundamental na teoria da computação, e por seu trabalho na quebra de códigos durante a Segunda Guerra Mundial, que ajudou a decifrar a máquina Enigma. Turing também fez contribuições significativas para a compreensão da computação e do que significa ser "inteligente".

Contexto:
Inteligência Artificial, com experiência no desenvolvimento , validação e monitoramento  de modelos de Machine Learning. Wesllei
também é pesquisador voluntário no Laboratório de Computação Móvel (Mobi lab) da Universidade do Vale do Rio dos Sinos
(UNISINOS), onde colabor a em pesquisas que envolvem a aplicação de tecnologias na área da saúde, principalmente envolvendo
Machine Learning. Atualmente pesquisa modelos de Machine Learning e Inteligência Artificial Gener ativa para a área de Saúde
Mental